In [1]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

Saving long_data_.csv to long_data_ (1).csv


In [2]:
df = pd.read_csv("long_data_.csv")
df.head()

,States,Regions,latitude,longitude,Dates,Usage
0,Punjab,NR,31.519974,75.980003,02/01/2019 00:00:00,119.9
1,Haryana,NR,28.450006,77.019991,02/01/2019 00:00:00,130.3
2,Rajasthan,NR,26.449999,74.639981,02/01/2019 00:00:00,234.1
3,Delhi,NR,28.669993,77.230004,02/01/2019 00:00:00,85.8
4,UP,NR,27.599981,78.050006,02/01/2019 00:00:00,313.9


In [3]:
# Convert 'Dates' column to datetime format
df["ds"] = pd.to_datetime(df["Dates"], format="%d/%m/%Y %H:%M:%S")

In [4]:
# Rename columns
df.rename(columns={"States": "unique_id", "Usage": "y"}, inplace=True)

In [5]:
# Keep only relevant columns
demand_df = df[["unique_id", "ds", "y"]].sort_values(["unique_id", "ds"]).reset_index(drop=True)


In [6]:
# Show sample
demand_df.head()

,unique_id,ds,y
0,Andhra Pradesh,2019-01-02,164.6
1,Andhra Pradesh,2019-01-03,170.1
2,Andhra Pradesh,2019-01-04,165.2
3,Andhra Pradesh,2019-01-05,167.4
4,Andhra Pradesh,2019-01-06,171.2


In [7]:
# Save the processed dataset
demand_df.to_csv("indian_electricity_demand.csv", index=False)


In [8]:
from google.colab import files
files.download("indian_electricity_demand.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
%pip install nixtla

In [10]:
%pip install --upgrade pip
%pip install nixtla utilsforecast mlforecast lightgbm scikit-learn pandas matplotlib


In [11]:
import os
os.environ['NIXTLA_API_KEY'] = 'nixak-JCA5ab7YJEHMivpmUlwl8NXhv8JgkglnSgEDCI4GWsTFKQtpQNr4TMDLWeedQZPEybgVp7t8O6PMXG0x'

In [12]:
from nixtla import NixtlaClient
nixtla_client = NixtlaClient(api_key=os.environ.get('NIXTLA_API_KEY'))

In [13]:
import pandas as pd
df = pd.read_csv('/content/indian_electricity_demand.csv')
df = df[['unique_id','ds','y']].sort_values(['unique_id','ds'])
df.info()
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/indian_electricity_demand.csv'

In [ ]:
import pandas as pd

df['ds'] = pd.to_datetime(df['ds'], errors='coerce')

print(df['ds'].dtypes)
print(df['ds'].head())


In [ ]:
freqs = (
    df.groupby('unique_id')['ds']
      .apply(lambda s: s.sort_values().dropna().diff().mode().iloc[0])
)
print(freqs.head())


In [ ]:
from nixtla import NixtlaClient
import os

os.environ['NIXTLA_API_KEY'] = "nixak-JCA5ab7YJEHMivpmUlwl8NXhv8JgkglnSgEDCI4GWsTFKQtpQNr4TMDLWeedQZPEybgVp7t8O6PMXG0x"


nixtla_client = NixtlaClient(api_key=os.environ['NIXTLA_API_KEY'])

print("Nixtla client created successfully:", nixtla_client)


In [ ]:
nixtla_client.plot(df, max_insample_length=365, time_col='ds', target_col='y')

In [ ]:
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(['unique_id', 'ds'])
print(df.duplicated(subset=['unique_id', 'ds']).sum())


In [ ]:
df = df.drop_duplicates(subset=['unique_id', 'ds'])

In [ ]:
filled_dfs = []
for uid, group in df.groupby('unique_id'):
    group = group.set_index('ds').asfreq('D')
    group['unique_id'] = uid
    group['y'] = group['y'].interpolate()
    filled_dfs.append(group.reset_index())
df = pd.concat(filled_dfs)


In [ ]:
freqs = df.groupby('unique_id')['ds'].apply(lambda s: s.sort_values().diff().mode().iloc[0])
print(freqs.head())

In [ ]:
anomalies_df = nixtla_client.detect_anomalies(
    df,
    time_col='ds',
    target_col='y',
    freq='D'
)
anomalies_df[anomalies_df['anomaly']==1].head()


In [ ]:
T1_df = df[df['unique_id']=='Andhra Pradesh'].sort_values('ds')

h = 30
test_df = T1_df.tail(h)
train_df = T1_df.iloc[:-h]

predict_df = nixtla_client.forecast(
    df = train_df,
    h = h,
    level = [90],
    finetune_steps = 60,
    finetune_loss = 'mae',
    model = 'timegpt-1-long-horizon',
    time_col = 'ds',
    target_col = 'y',
    freq = 'D'
)


In [ ]:
nixtla_client.plot(test_df, predict_df, models=['TimeGPT'], level=[90], time_col='ds', target_col='y')

In [ ]:
from utilsforecast.losses import mae, rmse, smape
from utilsforecast.evaluation import evaluate

predict_df['ds'] = pd.to_datetime(predict_df['ds'])
merged = pd.merge(test_df, predict_df, 'left', ['ds','unique_id'])

evaluation = evaluate(
    merged,
    metrics=[mae, rmse, smape],
    models=['TimeGPT'],
    target_col='y',
    id_col='unique_id'
)
print(evaluation)
avg_metrics = evaluation.groupby('metric')['TimeGPT'].mean()
print(avg_metrics)


In [ ]:
h = 30
test_df = df.groupby('unique_id').tail(h)
train_df = df.groupby('unique_id').apply(lambda g: g.iloc[:-h]).reset_index(drop=True)

predict_df = nixtla_client.forecast(
    df = train_df,
    h = h,
    level = [90],
    finetune_steps = 60,
    finetune_loss = 'mae',
    model = 'timegpt-1-long-horizon',
    time_col = 'ds',
    target_col = 'y',
    id_col = 'unique_id',
    freq = 'D'
)
nixtla_client.plot(test_df, predict_df, models=['TimeGPT'], level=[90], time_col='ds', target_col='y')


In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

h = 30

train_dfs = []
test_dfs = []

for uid, g in df.groupby('unique_id'):
    train_dfs.append(g.iloc[:-h])
    test_dfs.append(g.iloc[-h:])

train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)


In [ ]:
def create_features(data, lags=[1,2,3,7]):
    df_feat = data.copy()
    df_feat = df_feat.sort_values(['unique_id','ds'])

    df_feat['dayofweek'] = df_feat['ds'].dt.dayofweek
    df_feat['month'] = df_feat['ds'].dt.month
    df_feat['day'] = df_feat['ds'].dt.day

    for lag in lags:
        df_feat[f'lag_{lag}'] = df_feat.groupby('unique_id')['y'].shift(lag)

    df_feat['roll_mean_7'] = df_feat.groupby('unique_id')['y'].shift(1).rolling(7).mean().reset_index(level=0, drop=True)

    return df_feat

train_feat = create_features(train_df)
test_feat = create_features(pd.concat([train_df, test_df]))
test_feat = test_feat[test_feat['ds'].isin(test_df['ds'])]

train_feat = train_feat.dropna()
test_feat = test_feat.dropna()


In [ ]:
features = [col for col in train_feat.columns if col not in ['unique_id','ds','y']]

preds_list = []

for uid in df['unique_id'].unique():
    X_train = train_feat[train_feat['unique_id']==uid][features]
    y_train = train_feat[train_feat['unique_id']==uid]['y']

    X_test = test_feat[test_feat['unique_id']==uid][features]
    y_test = test_feat[test_feat['unique_id']==uid]['y']

    if len(y_train) > 20:
        model = LGBMRegressor(random_state=42, n_estimators=200)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        temp = test_feat[test_feat['unique_id']==uid][['unique_id','ds','y']].copy()
        temp['LGBM'] = y_pred
        preds_list.append(temp)

lgbm_pred_df = pd.concat(preds_list)


In [ ]:
comparison_df = lgbm_pred_df.merge(predict_df[['unique_id','ds','TimeGPT']], on=['unique_id','ds'], how='left')

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

results = []
for uid, g in comparison_df.groupby('unique_id'):
    mae_lgbm = mean_absolute_error(g['y'], g['LGBM'])
    rmse_lgbm = np.sqrt(mean_squared_error(g['y'], g['LGBM']))
    smape_lgbm = smape(g['y'], g['LGBM'])

    mae_tgpt = mean_absolute_error(g['y'], g['TimeGPT'])
    rmse_tgpt = np.sqrt(mean_squared_error(g['y'], g['TimeGPT']))
    smape_tgpt = smape(g['y'], g['TimeGPT'])

    results.append([uid, mae_lgbm, rmse_lgbm, smape_lgbm, mae_tgpt, rmse_tgpt, smape_tgpt])

results_df = pd.DataFrame(results, columns=['unique_id','LGBM_MAE','LGBM_RMSE','LGBM_SMAPE','TimeGPT_MAE','TimeGPT_RMSE','TimeGPT_SMAPE'])
print(results_df.head())

print("\nAverage performance:")
print(results_df.mean(numeric_only=True))


In [ ]:
import matplotlib.pyplot as plt

uid = 'Andhra Pradesh'
subset = comparison_df[comparison_df['unique_id']==uid]

plt.figure(figsize=(10,5))
plt.plot(subset['ds'], subset['y'], label='Actual')
plt.plot(subset['ds'], subset['LGBM'], label='LGBM Forecast')
plt.plot(subset['ds'], subset['TimeGPT'], label='TimeGPT Forecast')
plt.legend()
plt.title(f'Forecast Comparison - {uid}')
plt.xlabel('Date')
plt.ylabel('Electricity Demand')
plt.show()
